In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer,StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import operator

In [2]:
x_train = pd.read_csv('../data/pre_x_train.csv',parse_dates=['timestamp'])
y_train = pd.read_csv('../data/pre_y_train.csv')

x_test = pd.read_csv('../data/pre_x_test.csv',parse_dates=['timestamp'])
x_test_id = pd.read_csv('../data/pre_x_test.csv',usecols=['id'])

macro = pd.read_csv('../data/pre_macro.csv',parse_dates=['timestamp'])

In [3]:
x_train.drop(['id'], axis=1, inplace=True)
x_test.drop(['id'], axis=1, inplace=True)

# Build df_all = (x_train+x_test).join(macro)
num_train = len(x_train)
df_all = pd.concat([x_train, x_test])
df_all = pd.merge(df_all, macro, on='timestamp', how='left',sort=True)
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp'], axis=1, inplace=True)

(38133, 66)


In [4]:
df_all['rel_floor'] = df_all['floor'].values / df_all['max_floor'].values
df_all['rel_kitch_sq'] = df_all['kitch_sq'].values / df_all['full_sq'].values

In [5]:
x_train = df_all[:num_train]
x_test = df_all[num_train:]
print x_train.shape,x_test.shape

(30471, 71) (7662, 71)


In [6]:
for f in x_train.columns:
    if x_train[f].dtype==object:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[f].values)+list(x_train[f].values)) 
        x_train[f] = lbl.transform(list(x_train[f].values))
        x_test[f] = lbl.transform(list(x_test[f].values))

/home/jarvis/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jarvis/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
x_train1, x_valid1, y_train1, y_valid1 = train_test_split(x_train, y_train, test_size=0.3, random_state=4242)

In [8]:
print x_train.shape,x_test.shape

(30471, 71) (7662, 71)


In [9]:
params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['eta'] = 0.02
params['max_depth'] = 5
d_train = xgb.DMatrix(x_train1,y_train1)
d_valid = xgb.DMatrix(x_valid1,y_valid1)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, verbose_eval=100)

[0]	train-rmse:14.8169	valid-rmse:14.8261
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.02535	valid-rmse:2.03314
[200]	train-rmse:0.525781	valid-rmse:0.540645
[300]	train-rmse:0.442089	valid-rmse:0.464161
[400]	train-rmse:0.430167	valid-rmse:0.459795
[500]	train-rmse:0.422142	valid-rmse:0.458394
[600]	train-rmse:0.415712	valid-rmse:0.457371
[700]	train-rmse:0.409536	valid-rmse:0.456944
[800]	train-rmse:0.403048	valid-rmse:0.456715
[900]	train-rmse:0.396655	valid-rmse:0.456549
Stopping. Best iteration:
[862]	train-rmse:0.399053	valid-rmse:0.456505



In [10]:
sorted_x = sorted(bst.get_fscore().items(), key=operator.itemgetter(1))
sorted_x = sorted_x[::-1]
sorted_x = sorted_x[:50]

In [11]:
new = pd.DataFrame()
new1 = pd.DataFrame()
for i in range(0,len(sorted_x)):
    new[sorted_x[i][0]] = x_train[sorted_x[i][0]]
    new1[sorted_x[i][0]] = x_test[sorted_x[i][0]]

new.to_csv('../data/x_train_50.csv',index=False)
new1.to_csv('../data/x_test_50.csv',index=False)